In [25]:
import pandas as pd
from creditcard_mod import helpers
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBClassifier
# ignorer les FuturWarnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
df = helpers.load_csv("../datasets/data.csv")
df.head(3)

,step,customer,age,gender,zipcodeOri,merchant,zipMerchant,category,amount,fraud
0,0,1093826151,4,M,'28007',348934600,'28007',transportation,4.55,0
1,0,352968107,2,M,'28007',348934600,'28007',transportation,39.68,0
2,0,2054744914,4,F,'28007',1823072687,'28007',transportation,26.89,0


In [13]:
def make_mystep(df):
    df['step'] = 1577836800 + df['step'] * 3600 * 24
    df['step'] = pd.to_datetime(df['step'], unit='s')
    return df

In [24]:
test = make_mystep(df)
test.head()

,step,customer,age,gender,zipcodeOri,merchant,zipMerchant,category,amount,fraud
0,2020-01-01,1093826151,4,M,'28007',348934600,'28007',transportation,4.55,0
1,2020-01-01,352968107,2,M,'28007',348934600,'28007',transportation,39.68,0
2,2020-01-01,2054744914,4,F,'28007',1823072687,'28007',transportation,26.89,0
3,2020-01-01,1760612790,3,M,'28007',348934600,'28007',transportation,17.25,0
4,2020-01-01,757503768,5,M,'28007',348934600,'28007',transportation,35.72,0


In [14]:
def encoder(data):
    cat_cols = ['age', 'gender', 'category']
    enc = LabelEncoder()
    for col in cat_cols:
        data[col] = enc.fit_transform(data[col])
    return data

In [15]:
compute_features = {
    "nbre_trans_7jrs": 7,
    "nbre_trans_15jrs": 15,
    "nbre_trans_30jrs": 30
}
compute_features_cm = {
    "count_cust_merch_1_day":1,
    "count_cust_merch_7_day":7,
    "count_cust_merch_15_day":15
}

# Historique de transaction du client

def create_transaction_customer_historic(data):
    for key, value in compute_features.items():
        temp = pd.Series(data.index, index = data.step, name=key).sort_index()
        count_day = temp.rolling(str(value)+'d').count() -1
        count_day.index = temp.values
        data[key] = count_day.reindex(data.index)
    return data

# Historique de transaction du client sur un même marchant
def create_transaction_customer_merchant_historic(data):
    for key, value in compute_features_cm.items():
        temp = pd.Series(data.index, index = data.step, name=key).sort_index()
        count_day = temp.rolling(str(value)+'d').count() - 1
        count_day.index = temp.values
        data[key] = count_day.reindex(data.index)
    return data

In [16]:
def encoder(data):
    cat_cols = ['age', 'gender', 'category']
    enc = LabelEncoder()
    for col in cat_cols:
        data[col] = enc.fit_transform(data[col])
    return data

In [17]:
def pipeline_transform(data_init):
    df = make_mystep(data_init)
    df = df.groupby(['customer','merchant']).apply(create_transaction_customer_merchant_historic)
    df = df.groupby('customer').apply(create_transaction_customer_historic)
    df = df.drop(['customer','merchant','step'],axis=1)
    df = encoder(df)
    
    return df
    

In [6]:
#data = pipeline_transform(df)
#data.head()

In [18]:
import pickle

In [19]:
def predict(config, model):
    if type(config) == dict:
        df = pd.DataFrame(config)
    else:
        df = config
    data = pipeline_transform(df)
    y_pred = model.predict(data)
    return y_pred

In [4]:
data = {
    'step': 1,
    'age':4,
    'customer':"1093826151",
    'gender':2,
    'category':2,
    'amount' : 245,
    'merchant':"348934600"
    
}

In [21]:
with open('classifier.pkl', 'rb') as file:
        classifier = pickle.load(file)
        file.close()
prediction = predict(data, classifier)

In [5]:
import requests
url = "http://127.0.0.1:5000/Prediction/pred"
result = requests.post(url, json=data)
print(result.text)

{
  "fraud": 1
}

